In [2]:
import fitz
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
del1 = '../example_pdfs/delonghi/ESAM04110B-167807.pdf'
del2 = '../example_pdfs/delonghi/ECAM45076T-231470.pdf'

In [5]:
russel = '../example_pdfs/different_manufacturers/N25610-56_9707_11_OCT19_Russelhobbs.pdf'

In [3]:
melitta = '../example_pdfs/different_manufacturers/Mel_014_LatticiaOT_10_de-lt_WebZIlXIJSPMxbDw_Melitta.pdf'

In [6]:
doc = fitz.open(melitta)

In [8]:
fonts = []
sizes = []

In [11]:
passages = []

last_span = 0   # flag for what the last span was (0 = paragraph, 1 = h1, 2 = h2)
last_font = ""
last_size = 0
last_upper = False

for page in doc.pages():
    text = page.get_text('dict')

    for idx, block in enumerate(text['blocks']):
        if 'lines' in block:
            for line in block['lines']:
                for span in line['spans']:
                    span_text = span['text']
                    span_font = span['font']
                    span_size = span['size']
                    span_flags = flags_decomposer(span['flags'])
                    
                    span_text_upper = span_text.isupper()
                    
                    span_dic = {}

                    if last_font == span_font and last_size == span_size and last_upper == span_text_upper:
                        passages[-1]['paragraph'] = passages[-1]['paragraph'] + " " + span_text
                        continue
                    else:
                        span_dic['paragraph'] = span_text
                                     
                    span_dic['font'] = span_font
                    span_dic['size'] = span_size
                    span_dic['flags'] = span_flags
                    span_dic['uppercase'] = span_text_upper
                    
                    last_font = span_font
                    last_size = span_size
                    last_upper = span_text_upper
                    
                    passages.append(span_dic)

In [12]:
df_passages = pd.DataFrame(passages)

In [24]:
df_passages

,paragraph,font,size,flags,uppercase
0,Bedienungsanleitung / Operating Instructions /...,ArialUnicodeMS,11.0,"sans, proportional",False
1,DE GB FR NL IT ES DK SE NO RU PL LT,"Arial,Bold",8.0,"sans, proportional, bold",True
2,3 Inhaltsverzeichnis,MyriadPro-Light,9.0,"serifed, proportional",False
3,DE,"Arial,Bold",8.0,"sans, proportional, bold",True
4,1 Zu Ihrer Sicherheit,MyriadPro-Semibold,9.0,"serifed, proportional, bold",False
...,...,...,...,...,...
4322,Melitta Europa GmbH & Co. KG Division Househo...,MyriadPro-Regular,9.0,"serifed, proportional",False
4323,ZINDEL AG,MyriadPro-Regular,9.0,"serifed, proportional",True
4324,www.zindel.de,MyriadPro-Regular,9.0,"serifed, proportional",False
4325,MEL_014_Latticia_10_de-gb-fr-nl-it-es-dk-se-no...,MyriadPro-Light,7.0,"serifed, proportional",False


In [14]:
df_passages[df_passages['size'] == 11.5]

,paragraph,font,size,flags,uppercase
30,1.1 Bestimmungsgemäße Verwendung,MyriadPro-Semibold,11.5,"serifed, proportional, bold",False
31,Das Gerät ist für den Gebrauch im privaten Ha...,MyriadPro-Regular,11.5,"serifed, proportional",False
32,1.2 Gefahr durch elektrischen Strom,MyriadPro-Semibold,11.5,"serifed, proportional, bold",False
33,"Verwenden Sie das Gerät nur, wenn es in techn...",MyriadPro-Regular,11.5,"serifed, proportional",False
34,1.3 Verbrennungs- und Verbrühungs gefahr,MyriadPro-Semibold,11.5,"serifed, proportional, bold",False
...,...,...,...,...,...
4199,7.5 Programa „Valymas“,MyriadPro-Semibold,11.5,"serifed, proportional, bold",False
4222,7.6 Programa „Kalkių šalinimas“,MyriadPro-Semibold,11.5,"serifed, proportional, bold",False
4252,7.7 Vandens filtras,MyriadPro-Semibold,11.5,"serifed, proportional, bold",False
4274,7.8 Gamyklinės nuostatos,MyriadPro-Semibold,11.5,"serifed, proportional, bold",False


In [289]:
df_passages.groupby(['font', 'size', 'flags', 'uppercase']).count().sort_values(by=['paragraph'], ascending=False).reset_index()

,font,size,flags,uppercase,paragraph
0,MyriadPro-Regular,9.0,"serifed, proportional",False,1522
1,MyriadPro-It,9.0,"italic, serifed, proportional",False,409
2,MyriadPro-Semibold,11.5,"serifed, proportional, bold",False,372
3,Wingdings-Regular,9.0,"sans, proportional",False,312
4,MyriadPro-Semibold,9.0,"serifed, proportional, bold",False,274
5,MyriadPro-Regular,9.0,"superscript, serifed, proportional",False,273
6,MyriadPro-It,9.0,"superscript, italic, serifed, proportional",False,236
7,"Arial,Bold",8.0,"sans, proportional, bold",True,185
8,MyriadPro-Light,9.0,"serifed, proportional",False,184
9,MyriadPro-Semibold,12.0,"serifed, proportional, bold",False,144


In [95]:
df_passages = pd.DataFrame(passages)
determine_document_values(df_passages)

{'paragraph': {'font': 'MyriadPro-Cond', 'size': 9.0, 'flags': 4},
 'header1': {'font': 'MyriadPro-BoldCond', 'size': 10.0, 'flags': 20},
 'header12': {'font': 'MyriadPro-BoldCondIt', 'size': 9.0, 'flags': 22}}

In [10]:
def flags_decomposer(flags):
    """Make font flags human readable."""
    line = []    
    if flags & 2 ** 0:
        line.append("superscript")
    if flags & 2 ** 1:
        line.append("italic")
    if flags & 2 ** 2:
        line.append("serifed")
    else:
        line.append("sans")
    if flags & 2 ** 3:
        line.append("monospaced")
    else:
        line.append("proportional")
    if flags & 2 ** 4:
        line.append("bold")
    return ", ".join(line)

In [96]:
flags_decomposer(22)

'italic, serifed, proportional, bold'

In [85]:
def determine_document_values(frame):
    df_counts = frame.groupby(['font', 'size', 'flags']).count().sort_values(by=['paragraph'], ascending=False).reset_index()
    
    top8 = df_counts.iloc[0:7]
    top2_bold = df_counts.iloc[1:6].loc[res['font'].str.contains('Bold')].iloc[0:2].sort_values(by=['size'], ascending=True)
    
    document_values = {}
    
    paragraph_dic = {}
    first_entry = top8.iloc[0] 
    paragraph_dic['font'] = first_entry['font']
    paragraph_dic['size'] = first_entry['size']
    paragraph_dic['flags'] = first_entry['flags']
    
    
    header1_dic = {}
    first_entry = top2_bold.iloc[1]
    header1_dic['font'] = first_entry['font']
    header1_dic['size'] = first_entry['size']
    header1_dic['flags'] = first_entry['flags']
    
    header2_dic = {}
    first_entry = top2_bold.iloc[0]
    header2_dic['font'] = first_entry['font']
    header2_dic['size'] = first_entry['size']
    header2_dic['flags'] = first_entry['flags']
    
    document_values['paragraph'] = paragraph_dic
    document_values['header1'] = header1_dic
    document_values['header12'] = header2_dic
    
    return document_values

In [87]:
determine_document_values(df_passages)

{'paragraph': {'font': 'MyriadPro-Cond', 'size': 9.0, 'flags': 4},
 'header1': {'font': 'MyriadPro-BoldCond', 'size': 11.0, 'flags': 20},
 'header12': {'font': 'MyriadPro-BoldCond', 'size': 9.0, 'flags': 20}}

In [16]:
flags_decomposer(6)

'italic, serifed, proportional'

In [61]:
res = df_passages.groupby(['font', 'size', 'flags']).count().sort_values(by=['paragraph'], ascending=False).reset_index()

In [63]:
res

,font,size,flags,paragraph
0,MyriadPro-Cond,9.000000,4,118
1,MyriadPro-BoldCond,9.000000,20,51
2,MyriadPro-CondIt,9.000000,6,47
3,MyriadPro-BoldCond,11.000000,20,20
4,Helvetica-Condensed-Bold,7.848800,20,11
5,Helvetica-Condensed-Bold,8.134200,20,11
6,MyriadPro-Cond,8.000000,4,7
7,MyriadPro-Cond,9.117631,4,6
8,MyriadPro-BoldCond,9.117631,20,2
9,MyriadPro-Cond,13.000000,4,2


In [72]:
res.iloc[1:6]

,font,size,flags,paragraph
1,MyriadPro-BoldCond,9.0000,20,51
2,MyriadPro-CondIt,9.0000,6,47
3,MyriadPro-BoldCond,11.0000,20,20
4,Helvetica-Condensed-Bold,7.8488,20,11
5,Helvetica-Condensed-Bold,8.1342,20,11


In [75]:
top2_bold = res.iloc[1:6].loc[res['font'].str.contains('Bold')].iloc[0:2].sort_values(by=['size'], ascending=True)

In [77]:
top2_bold.iloc[0]

font         MyriadPro-BoldCond
size                        9.0
flags                        20
paragraph                    51
Name: 1, dtype: object

In [40]:
entry = res.sort_values(by=['paragraph'], ascending=False).iloc[0:]

In [79]:
determine_document_values(res)

{'paragraph': {'font': 'MyriadPro-Cond', 'size': 9.0, 'flags': 4},
 'header1': {'font': 'MyriadPro-BoldCond', 'size': 11.0, 'flags': 20},
 'header12': {'font': 'MyriadPro-BoldCond', 'size': 9.0, 'flags': 20}}

In [48]:
paragraph_font, paragraph_size, paragraph_flags = entry.name[0], entry.name[1], entry.name[2]

In [20]:
df_value_counts = df_passages[['font', 'size', 'flags']].value_counts()

In [23]:
df_value_counts.iloc[0]

118

In [10]:
df_passages['size'].value_counts(dropna=False)

9.000000     217
11.000000     20
7.848800      11
8.134200      11
9.117631       8
8.000000       7
13.000000      3
15.000000      1
8.519500       1
73.095001      1
9.126131       1
11.143810      1
2.069200       1
Name: size, dtype: int64

In [51]:
for page in doc.pages(3, len(list(doc.pages())), 1):
    text = page.get_text('dict')
    page_structure = []

    for idx, block in enumerate(text['blocks']):
        if 'lines' in block:
            for line in block['lines']:
                for span in line['spans']:
                    span_text = span['text']
                    span_font = span['font']
                    span_size = span['size']

                    fonts.append(span_font)
                    sizes.append(span_size)

In [52]:
df_fonts = pd.DataFrame(list(zip(fonts, sizes)), columns=['Fonts', 'Sizes'])

In [53]:
df_fonts

,Fonts,Sizes
0,MyriadPro-Cond,9.0
1,MyriadPro-BoldCond,11.0
2,MyriadPro-BoldCond,9.0
3,MyriadPro-Cond,9.0
4,MyriadPro-Cond,9.0
...,...,...
1325,MyriadPro-Cond,9.0
1326,MyriadPro-Cond,9.0
1327,MyriadPro-Cond,9.0
1328,MyriadPro-Cond,9.0


In [54]:
df_fonts['Fonts'].value_counts(dropna=False)

MyriadPro-Cond              973
Helvetica-Condensed-Bold    246
MyriadPro-BoldCond           65
MyriadPro-CondIt             39
Myriad-Roman                  5
MyriadPro-Regular             1
TimesNewRomanPSMT             1
Name: Fonts, dtype: int64

In [55]:
df_fonts['Sizes'].value_counts(dropna=False)

9.000000     1015
7.848800      121
8.134200      121
9.117631       44
11.000000      17
8.519500        5
9.126131        4
73.095001       1
11.143810       1
2.069200        1
Name: Sizes, dtype: int64

In [16]:
del1 = '../example_pdfs/delonghi/ESAM04110B-167807.pdf'
del1 = '../example_pdfs/delonghi/ECAM45076T-231470.pdf'
doc_test = fitz.open(del1)

In [213]:
p = doc_test.load_page(3)

In [161]:
p.number

1

In [28]:
page_text = doc_test.load_page(3).get_text()
# page_text = ''.join([i for i in page_text if not i.isdigit()])
len(page_text)

4857

In [29]:
page_text.count('.')

2404

In [30]:
sum(map(str.isdigit, page_text))

295

In [ ]:
passages = []

for page in doc.pages():
    text = page.get_text('dict')

    for idx, block in enumerate(text['blocks']):
        if 'lines' in block:
            for line in block['lines']:
                for span in line['spans']:
                    span_text = span['text']
                    span_font = span['font']
                    span_size = span['size']
                    span_flags = flags_decomposer(span['flags'])
                    
                    span_text_upper = span_text.isupper()
                    
                    span_dic = {}

                    if last_font == span_font and last_size == span_size and last_upper == span_text_upper:
                        passages[-1]['paragraph'] = passages[-1]['paragraph'] + " " + span_text
                        continue
                    else:
                        span_dic['paragraph'] = span_text
                                     
                    span_dic['font'] = span_font
                    span_dic['size'] = span_size
                    span_dic['flags'] = span_flags
                    span_dic['uppercase'] = span_text_upper
                    
                    last_font = span_font
                    last_size = span_size
                    last_upper = span_text_upper
                    
                    passages.append(span_dic)

In [251]:
import numpy as np

def detect_toc(doc):
    """Method to identify the ToC in a given pdf document.
    
    TODO: Properly identify it with looking at how much more occurences appear.
    TODO: What if document does not have ToC -> Certainty?

    Parameters
    ----------
    doc : _type_
        _description_
    """
    point_count = []
    number_count = []
    
    doc_pages = doc.pages()
    
    for page in doc_pages:
        text = page.get_text()

        point_count.append(text.count('.'))
        number_count.append(sum(map(str.isdigit, text)))
        
    np.array(point_count)
    np.array(number_count)
    
    idx = int(np.argmax(point_count))   
    toc = doc.load_page(idx)
    
    return toc

In [254]:
detect_toc(doc_test)

page 0 of ../example_pdfs/delonghi/ESAM04110B-167807.pdf

In [269]:
def get_relevant_pages(doc, cutoff=500):
    """Method to identify all relevant pages in a given pdf document.

    Parameters
    ----------
    doc : _type_
        _description_
    """
    relevant_pages = []
    
    for page in doc.pages():
        text = page.get_text()

        # don't add toc page to relevant pages
        if toc.number == page.number:
            continue
        
        if len(text) > cutoff:
            relevant_pages.append(page)
            
    return relevant_pages

In [273]:
rel_pages = get_relevant_pages(doc_test)

In [270]:
toc = detect_toc(doc_test)

In [277]:
toc.number

3

In [18]:
doc_test.load_page(5).get_text('dict')

{'width': 419.528,
 'height': 595.276,
 'blocks': [{'number': 0,
   'type': 0,
   'bbox': (207.9375,
    572.3511962890625,
    211.40249633789062,
    583.0161743164062),
   'lines': [{'spans': [{'size': 9.0,
       'flags': 4,
       'font': 'MyriadPro-Cond',
       'color': 2236191,
       'ascender': 0.9350000023841858,
       'descender': -0.25,
       'text': '6',
       'origin': (207.9375, 580.7661743164062),
       'bbox': (207.9375,
        572.3511962890625,
        211.40249633789062,
        583.0161743164062)}],
     'wmode': 0,
     'dir': (1.0, 0.0),
     'bbox': (207.9375,
      572.3511962890625,
      211.40249633789062,
      583.0161743164062)}]},
  {'number': 1,
   'type': 0,
   'bbox': (28.346500396728516,
    26.681922912597656,
    204.1047821044922,
    37.346923828125),
   'lines': [{'spans': [{'size': 9.0,
       'flags': 4,
       'font': 'MyriadPro-Cond',
       'color': 2236191,
       'ascender': 0.9350000023841858,
       'descender': -0.25,
       'tex